In [1]:
import numpy as np
import scipy as scp
import scipy.stats as ss
from scipy.integrate import quad
from functools import partial
import yfinance as yf
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from datetime import date
now = datetime.today().strftime('%Y:%m:%d')
d1 = datetime.strptime(now, '%Y:%m:%d')
stk = yf.Ticker("MSFT")
expiry = stk.options[6]         #expiry date choice
opt = stk.option_chain(expiry)  #selecting that option based on expiry date                         
low = opt.calls.iloc[10]          #selecting specific call
low1 = opt.puts.iloc[10]          #selecting specific put
d2 = datetime.strptime(expiry, '%Y-%m-%d')
diff = (d2 - d1).days
S0 = stk.info['open']
K= low.loc['strike']    # strike
T= diff/365      # maturity 
r= 0.011       # risk free rate 
sig = low.loc['impliedVolatility'] 

In [3]:
np.random.seed(seed=44)  # seed for random number generation
N = 10000000  # Number of random variables

W = ss.norm.rvs( (r-0.5*sig**2)*T , np.sqrt(T)*sig, N)
S_T = S0 * np.exp(W)

call = np.mean( np.exp(-r*T) * np.maximum(S_T-K,0) )
put = np.mean( np.exp(-r*T) * np.maximum(K-S_T,0) )
call_err = ss.sem( np.exp(-r*T) * np.maximum(S_T-K,0) )  # standard error
put_err = ss.sem( np.exp(-r*T) * np.maximum(K-S_T,0) )   # standard error
print("Call price: {}, with error: {}".format(call, call_err))
print("Put price: {}, with error: {}".format(put, put_err))

Call price: 10.493155277490384, with error: 0.005401560123135842
Put price: 11.593490705697974, with error: 0.00483595724265883


In [4]:
print("Difference between model and actual (call): ", (low.loc['lastPrice'] - call))
print("Difference between model and actual (put): ", (low1.loc['lastPrice'] - put))

Difference between model and actual (call):  0.20684472250961505
Difference between model and actual (put):  1.6665092943020259
